In [195]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [196]:
ib.client.reqMarketDataType( 3 )

In [120]:
def vixFutContract(localSym=None):
    contract = Contract()
    contract.symbol = "VIX"
    contract.exchange = "CFE"
    contract.secType = "FUT"
    contract.tradingClass = 'VX'
    contract.localSymbol = localSym

    return contract

In [140]:
contracts = []
contracts.append(vixFutContract('VXF9'))
eurusd = contracts[0]

In [141]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

In [142]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Contract(secType='FUT', symbol='VIX', exchange='CFE', localSymbol='VXF9', tradingClass='VX'), time=datetime.datetime(2018, 12, 19, 22, 33, 41, 133016, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=22.3, lastSize=8, volume=162951, high=22.55, low=20.85, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

In [143]:
ticker.marketPrice()

nan

In [216]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bid ask'.split())
df['symbol'] = [c.localSymbol for c in contracts if c.localSymbol]
df=df.set_index('symbol')
#contract2Row = {c: i for (i, c) in enumerate(contracts)}

In [218]:
def onPendingTickers(tickers):
    for t in tickers:
        if t.contract.localSymbol:
            row = (t.contract.localSymbol)
            print (row,t.bid, t.ask)
            df.loc[row] = (t.bid, t.ask)
        #clear_output(wait=True)
    #display(df) 

In [219]:
ib.pendingTickersEvent += onPendingTickers
#ib.sleep(30)
#ib.pendingTickersEvent -= onPendingTickers

In [135]:
for contract in contracts:
    ib.cancelMktData(contract)
ib.pendingTickersEvent -= onPendingTickers

In [117]:
ib.disconnect()

In [197]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)


[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

In [198]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2018, 12, 19, 22, 58, 25, 917836, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=2506.87, lastSize=0, high=2585.29, low=2488.96, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

In [199]:
spxValue = ticker.marketPrice()
spxValue

nan

In [200]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"{20190214, 20190314, 20200319, 20181220, 20201...","{2050.0, 4100.0, 2055.0, 2060.0, 2065.0, 2070...."
1,CBOE,416904,SPXW,100,"{20190102, 20190621, 20190114, 20181224, 20190...","{2050.0, 4100.0, 2055.0, 2060.0, 2065.0, 2070...."
2,SMART,416904,SPXW,100,"{20190102, 20190621, 20190114, 20181224, 20190...","{2050.0, 4100.0, 2055.0, 2060.0, 2065.0, 2070...."
3,CBOE,416904,SPX,100,"{20190214, 20190314, 20200319, 20181220, 20201...","{2050.0, 4100.0, 2055.0, 2060.0, 2065.0, 2070...."


In [201]:
chain=next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=416904, tradingClass='SPX', multiplier='100', expirations={'20190214', '20190314', '20200319', '20181220', '20201217', '20190620', '20191219', '20200618', '20200116', '20190117', '20190919'}, strikes={2050.0, 4100.0, 2055.0, 2060.0, 2065.0, 2070.0, 2075.0, 2080.0, 2085.0, 2090.0, 2095.0, 2100.0, 2105.0, 2110.0, 2115.0, 2120.0, 2125.0, 2130.0, 2135.0, 2140.0, 2145.0, 100.0, 2150.0, 4200.0, 2155.0, 2160.0, 2165.0, 2170.0, 2175.0, 2180.0, 2185.0, 2190.0, 2195.0, 2200.0, 2205.0, 2210.0, 2215.0, 2220.0, 2225.0, 2230.0, 2235.0, 2240.0, 2245.0, 200.0, 2250.0, 2255.0, 2260.0, 2265.0, 2270.0, 2275.0, 2280.0, 2285.0, 2290.0, 2295.0, 250.0, 2300.0, 2305.0, 2310.0, 2315.0, 2320.0, 2325.0, 2330.0, 2335.0, 2340.0, 2345.0, 300.0, 2350.0, 2355.0, 2360.0, 2365.0, 2370.0, 2375.0, 2380.0, 2385.0, 2390.0, 2395.0, 2400.0, 2405.0, 2410.0, 2415.0, 2420.0, 2425.0, 2430.0, 2435.0, 2440.0, 2445.0, 400.0, 2450.0, 2455.0, 2460.0, 2465.0, 2470.0, 2475.0, 2480.0, 2485.0

In [206]:
strikes = chain.strikes
expirations = sorted(exp for exp in chain.expirations if exp>'2019')[:1]
rights = ['P', 'C']

contracts = [esOptContract(expiration, strike,right)
        for right in rights
        for expiration in expirations
        for strike in sorted(strikes)]

#ib.qualifyContracts(*contracts)
len(contracts)

652

In [207]:
contracts[0]

Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=100.0, right='P', exchange='SMART')

In [192]:
ib.reqMktData(contracts[0], '', False, False)

Ticker(contract=Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2050.0, right='P', exchange='SMART'), ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

Error 10090, reqId 1320: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2050.0, right='P', exchange='SMART')


In [202]:
def esOptContract(expiration, strike,right):
    contract = Contract()
    contract.symbol = "SPX"
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.lastTradeDateOrContractMonth = expiration
    contract.strike=strike
    contract.right=right
    
    return contract

In [204]:
import time

In [208]:
for c in contracts:
    print (c.strike,c.right)
    ib.qualifyContracts(c)
    time.sleep(0.1)

100.0 P


Error 200, reqId 40: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=100.0, right='P', exchange='SMART')
Unknown contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=100.0, right='P', exchange='SMART')


200.0 P


Error 200, reqId 41: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=200.0, right='P', exchange='SMART')
Unknown contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=200.0, right='P', exchange='SMART')


250.0 P


...

Error 200, reqId 458: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2095.0, right='C', exchange='SMART')
Unknown contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2095.0, right='C', exchange='SMART')


2100.0 C
2105.0 C
2110.0 C
2115.0 C
2120.0 C
2125.0 C
2130.0 C
2135.0 C
2140.0 C
2145.0 C
2150.0 C
2155.0 C
2160.0 C
2165.0 C
2170.0 C
2175.0 C
2180.0 C
2185.0 C
2190.0 C
2195.0 C
2200.0 C
2205.0 C
2210.0 C
2215.0 C
2220.0 C
2225.0 C
2230.0 C
2235.0 C
2240.0 C
2245.0 C
2250.0 C
2255.0 C
2260.0 C
2265.0 C
2270.0 C
2275.0 C
2280.0 C
2285.0 C
2290.0 C
2295.0 C
2300.0 C
2305.0 C
2310.0 C
2315.0 C
2320.0 C
2325.0 C
2330.0 C
2335.0 C
2340.0 C
2345.0 C
2350.0 C
2355.0 C
2360.0 C
2365.0 C
2370.0 C
2375.0 C
2380.0 C
2385.0 C
2390.0 C
2395.0 C
2400.0 C
2405.0 C
2410.0 C
2415.0 C
2420.0 C
2425.0 C
2430.0 C
2435.0 C
2440.0 C
2445.0 C
2450.0 C
2455.0 C
2460.0 C
2465.0 C
2470.0 C
2475.0 C
2480.0 C
2485.0 C
2490.0 C
2495.0 C
2500.0 C
2505.0 C
2510.0 C
2515.0 C
2520.0 C
2525.0 C
2530.0 C
2535.0 C
2540.0 C
2545.0 C
2550.0 C
2555.0 C
2560.0 C
2565.0 C
2570.0 C
2575.0 C
2580.0 C
2585.0 C
2590.0 C
2595.0 C
2600.0 C
2605.0 C
2610.0 C
2615.0 C
2620.0 C
2625.0 C
2630.0 C
2635.0 C
2640.0 C
2645.0 C
2650.0 C
2

Error 200, reqId 690: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=4100.0, right='C', exchange='SMART')
Unknown contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=4100.0, right='C', exchange='SMART')


4200.0 C


Error 200, reqId 691: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=4200.0, right='C', exchange='SMART')
Unknown contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=4200.0, right='C', exchange='SMART')


In [209]:
ib.tickers()

[Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2018, 12, 19, 22, 58, 25, 917836, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=2506.87, lastSize=0, high=2585.29, low=2488.96, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])]

In [220]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)
    time.sleep(0.1)

KeyboardInterrupt: 

Error 200, reqId 1344: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=100.0, right='P', exchange='SMART')
Error 200, reqId 1345: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=200.0, right='P', exchange='SMART')
Error 200, reqId 1346: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=250.0, right='P', exchange='SMART')
Error 200, reqId 1347: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=300.0, right='P', exchange='SMART')
Error 200, reqId 1348: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='SPX', lastTradeDateOrContractMonth='2019

Error 10090, reqId 1365: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=345951637, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1050.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01050000', tradingClass='SPX')
Error 10090, reqId 1366: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=346047556, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1075.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01075000', tradingClass='SPX')
Error 10090, reqId 1373: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

Error 10090, reqId 1391: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290075992, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1700.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01700000', tradingClass='SPX')
Error 10090, reqId 1392: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290075997, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1725.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01725000', tradingClass='SPX')
Error 10090, reqId 1393: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

Error 10090, reqId 1413: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=337284195, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1940.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01940000', tradingClass='SPX')
Error 10090, reqId 1414: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290076032, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1950.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01950000', tradingClass='SPX')
Error 10090, reqId 1415: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

...

Error 10090, reqId 1798: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290075787, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2275.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02275000', tradingClass='SPX')
Error 10090, reqId 1799: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=334826273, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2280.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02280000', tradingClass='SPX')
Error 10090, reqId 1800: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

Error 10090, reqId 1818: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290075802, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2375.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02375000', tradingClass='SPX')
Error 10090, reqId 1819: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=334826429, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2380.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02380000', tradingClass='SPX')
Error 10090, reqId 1820: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

Error 10090, reqId 1838: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290075817, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2475.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02475000', tradingClass='SPX')
Error 10090, reqId 1839: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=334826642, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2480.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02480000', tradingClass='SPX')
Error 10090, reqId 1840: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

Error 10090, reqId 1858: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=290075832, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2575.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02575000', tradingClass='SPX')
Error 10090, reqId 1859: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Contract(secType='OPT', conId=334826847, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2580.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02580000', tradingClass='SPX')
Error 10090, reqId 1860: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data 

SPX   190118P02790000 -1.0 -1.0
SPX   190118P02900000 -1.0 -1.0
SPX   190118P02805000 -1.0 -1.0
SPX   190118C02000000 -1.0 -1.0
SPX   190118P02880000 -1.0 -1.0
SPX   190118P02625000 -1.0 -1.0
SPX   190118C01575000 -1.0 -1.0
SPX   190118C02385000 -1.0 -1.0
SPX   190118P02575000 -1.0 -1.0
SPX   190118P02530000 -1.0 -1.0
SPX   190118P02620000 -1.0 -1.0
SPX   190118P02665000 -1.0 -1.0
SPX   190118P02710000 -1.0 -1.0
SPX   190118P02755000 -1.0 -1.0
SPX   190118P02800000 -1.0 -1.0
SPX   190118P02845000 -1.0 -1.0
SPX   190118P03145000 -1.0 -1.0
SPX   190118P02935000 -1.0 -1.0
SPX   190118C01275000 -1.0 -1.0
SPX   190118P02905000 -1.0 -1.0
SPX   190118C02490000 -1.0 -1.0
SPX   190118C02520000 -1.0 -1.0
SPX   190118P03005000 -1.0 -1.0
SPX   190118P03095000 -1.0 -1.0
SPX   190118P03140000 -1.0 -1.0
SPX   190118C01250000 -1.0 -1.0
SPX   190118C01910000 -1.0 -1.0
SPX   190118C02050000 -1.0 -1.0
SPX   190118C01700000 -1.0 -1.0
SPX   190118C02410000 -1.0 -1.0
SPX   190118C02455000 -1.0 -1.0
SPX   19

In [222]:
1

1

In [223]:
ib.disconnect()